In [1]:
import os
import copy
import time
import numpy as np
from tqdm import tqdm
import torch

from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
from utils import get_dataset, average_weights, exp_details
import matplotlib
import matplotlib.pyplot as plt

# To fix potential issues with matplot lib
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [6]:
class Args:
  epochs = 10
  num_users = 100
  frac = 0.1
  local_ep = 10
  local_bs = 10
  lr = 0.1
  momentum = 0.5
  model = "mlp"
  num_channels = 1
  norm = "batch_norm"
  dataset = "fmnist"
  num_classes = 10
  optimizer = "sgd"
  iid = 1
  unequal = 0
  stopping_rounds = 10
  verbose = 1
  seed = 1
  gpu = None

In [7]:
start_time = time.time()

args = Args()
exp_details(args)

device = "cpu"

# load dataset and user groups
train_dataset, test_dataset, user_groups = get_dataset(args)


Experimental details:
    Model     : mlp
    Optimizer : sgd
    Learning  : 0.1
    Global Rounds   : 10

    Federated parameters:
    IID
    Fraction of users  : 0.1
    Local Batch size   : 10
    Local Epochs       : 10



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/fmnist/MNIST\raw\train-images-idx3-ubyte.gz to ../data/fmnist/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/fmnist/MNIST\raw\train-labels-idx1-ubyte.gz to ../data/fmnist/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/fmnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ../data/fmnist/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/fmnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data/fmnist/MNIST\raw



In [8]:
# BUILD MODEL
if args.model == 'cnn':
    # Convolutional neural netork
    if args.dataset == 'mnist':
        global_model = CNNMnist(args=args)
    elif args.dataset == 'fmnist':
        global_model = CNNFashion_Mnist(args=args)
    elif args.dataset == 'cifar':
        global_model = CNNCifar(args=args)

elif args.model == 'mlp':
    # Multi-layer preceptron
    img_size = train_dataset[0][0].shape
    len_in = 1
    for x in img_size:
        len_in *= x
        global_model = MLP(dim_in=len_in, dim_hidden=64,
                           dim_out=args.num_classes)
        
print(global_model)        

MLP(
  (layer_input): Linear(in_features=784, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [9]:
# Set the model to train and send it to device.
global_model.to(device)
global_model.train()

# copy weights
global_weights = global_model.state_dict()

# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

In [10]:
print("Running training for {} global epochs and {} local epochs".format(args.epochs, args.local_ep))

Running training for 10 global epochs and 10 local epochs


In [ ]:
for epoch in range(args.epochs):
    # init local weights and loss
    local_weights, local_losses = [], []
    print(f'Global Training Round : {epoch+1}\n')

    global_model.train()
    # sample a fraction of users (with args frac)
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    print("Devices selected in global round {}: {}".format(epoch+1,idxs_users))

    # for each sampled user
    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], device_id=idx)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
        
    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], device_id=c)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))
    
    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f'\nAvg Training Stats after {epoch+1} global rounds:')
        print('Training Loss: {:.3f}'.format(np.mean(np.array(train_loss))))
        print('Train Accuracy: {:.2f}%\n'.format(100*train_accuracy[-1]))

Global Training Round : 1

Devices selected in global round 1: [30 76 41 82 75  5 17 39 93 70]
| Global Round : 0 | Local Epoch : 0 on device 30 | Images on device 30 [000/480 (0%)] Loss: -0.106
| Global Round : 0 | Local Epoch : 0 on device 30 | Images on device 30 [100/480 (21%)] Loss: -0.112
| Global Round : 0 | Local Epoch : 0 on device 30 | Images on device 30 [200/480 (42%)] Loss: -0.107
| Global Round : 0 | Local Epoch : 0 on device 30 | Images on device 30 [300/480 (62%)] Loss: -0.452
| Global Round : 0 | Local Epoch : 0 on device 30 | Images on device 30 [400/480 (83%)] Loss: -0.329
| Global Round : 0 | Local Epoch : 1 on device 30 | Images on device 30 [000/480 (0%)] Loss: -0.605
| Global Round : 0 | Local Epoch : 1 on device 30 | Images on device 30 [100/480 (21%)] Loss: -0.501


E:\Mestrado\ENSF619.02\ENSF_619_02_Tutorial_Group_9\update.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)


| Global Round : 0 | Local Epoch : 1 on device 30 | Images on device 30 [200/480 (42%)] Loss: -0.799
| Global Round : 0 | Local Epoch : 1 on device 30 | Images on device 30 [300/480 (62%)] Loss: -0.620
| Global Round : 0 | Local Epoch : 1 on device 30 | Images on device 30 [400/480 (83%)] Loss: -0.790
| Global Round : 0 | Local Epoch : 2 on device 30 | Images on device 30 [000/480 (0%)] Loss: -0.508
| Global Round : 0 | Local Epoch : 2 on device 30 | Images on device 30 [100/480 (21%)] Loss: -0.640
| Global Round : 0 | Local Epoch : 2 on device 30 | Images on device 30 [200/480 (42%)] Loss: -0.661
| Global Round : 0 | Local Epoch : 2 on device 30 | Images on device 30 [300/480 (62%)] Loss: -0.831
| Global Round : 0 | Local Epoch : 2 on device 30 | Images on device 30 [400/480 (83%)] Loss: -0.611
| Global Round : 0 | Local Epoch : 3 on device 30 | Images on device 30 [000/480 (0%)] Loss: -0.587
| Global Round : 0 | Local Epoch : 3 on device 30 | Images on device 30 [100/480 (21%)] Loss:

| Global Round : 0 | Local Epoch : 8 on device 76 | Images on device 76 [100/480 (21%)] Loss: -0.850
| Global Round : 0 | Local Epoch : 8 on device 76 | Images on device 76 [200/480 (42%)] Loss: -0.789
| Global Round : 0 | Local Epoch : 8 on device 76 | Images on device 76 [300/480 (62%)] Loss: -0.765
| Global Round : 0 | Local Epoch : 8 on device 76 | Images on device 76 [400/480 (83%)] Loss: -0.779
| Global Round : 0 | Local Epoch : 9 on device 76 | Images on device 76 [000/480 (0%)] Loss: -0.620
| Global Round : 0 | Local Epoch : 9 on device 76 | Images on device 76 [100/480 (21%)] Loss: -0.871
| Global Round : 0 | Local Epoch : 9 on device 76 | Images on device 76 [200/480 (42%)] Loss: -0.770
| Global Round : 0 | Local Epoch : 9 on device 76 | Images on device 76 [300/480 (62%)] Loss: -0.902
| Global Round : 0 | Local Epoch : 9 on device 76 | Images on device 76 [400/480 (83%)] Loss: -0.862
| Global Round : 0 | Local Epoch : 0 on device 41 | Images on device 41 [000/480 (0%)] Loss:

| Global Round : 0 | Local Epoch : 5 on device 82 | Images on device 82 [200/480 (42%)] Loss: -0.799
| Global Round : 0 | Local Epoch : 5 on device 82 | Images on device 82 [300/480 (62%)] Loss: -0.692
| Global Round : 0 | Local Epoch : 5 on device 82 | Images on device 82 [400/480 (83%)] Loss: -0.797
| Global Round : 0 | Local Epoch : 6 on device 82 | Images on device 82 [000/480 (0%)] Loss: -0.796
| Global Round : 0 | Local Epoch : 6 on device 82 | Images on device 82 [100/480 (21%)] Loss: -0.781
| Global Round : 0 | Local Epoch : 6 on device 82 | Images on device 82 [200/480 (42%)] Loss: -0.711
| Global Round : 0 | Local Epoch : 6 on device 82 | Images on device 82 [300/480 (62%)] Loss: -0.697
| Global Round : 0 | Local Epoch : 6 on device 82 | Images on device 82 [400/480 (83%)] Loss: -0.891
| Global Round : 0 | Local Epoch : 7 on device 82 | Images on device 82 [000/480 (0%)] Loss: -0.769
| Global Round : 0 | Local Epoch : 7 on device 82 | Images on device 82 [100/480 (21%)] Loss:

...

| Global Round : 1 | Local Epoch : 2 on device 89 | Images on device 89 [400/480 (83%)] Loss: -0.550
| Global Round : 1 | Local Epoch : 3 on device 89 | Images on device 89 [000/480 (0%)] Loss: -0.895
| Global Round : 1 | Local Epoch : 3 on device 89 | Images on device 89 [100/480 (21%)] Loss: -0.607
| Global Round : 1 | Local Epoch : 3 on device 89 | Images on device 89 [200/480 (42%)] Loss: -0.947
| Global Round : 1 | Local Epoch : 3 on device 89 | Images on device 89 [300/480 (62%)] Loss: -0.840
| Global Round : 1 | Local Epoch : 3 on device 89 | Images on device 89 [400/480 (83%)] Loss: -0.693
| Global Round : 1 | Local Epoch : 4 on device 89 | Images on device 89 [000/480 (0%)] Loss: -0.769
| Global Round : 1 | Local Epoch : 4 on device 89 | Images on device 89 [100/480 (21%)] Loss: -0.904
| Global Round : 1 | Local Epoch : 4 on device 89 | Images on device 89 [200/480 (42%)] Loss: -0.854
| Global Round : 1 | Local Epoch : 4 on device 89 | Images on device 89 [300/480 (62%)] Loss:

| Global Round : 1 | Local Epoch : 9 on device 22 | Images on device 22 [200/480 (42%)] Loss: -0.802
| Global Round : 1 | Local Epoch : 9 on device 22 | Images on device 22 [300/480 (62%)] Loss: -0.898
| Global Round : 1 | Local Epoch : 9 on device 22 | Images on device 22 [400/480 (83%)] Loss: -0.928
| Global Round : 1 | Local Epoch : 0 on device 63 | Images on device 63 [000/480 (0%)] Loss: -0.763
| Global Round : 1 | Local Epoch : 0 on device 63 | Images on device 63 [100/480 (21%)] Loss: -0.736
| Global Round : 1 | Local Epoch : 0 on device 63 | Images on device 63 [200/480 (42%)] Loss: -0.603
| Global Round : 1 | Local Epoch : 0 on device 63 | Images on device 63 [300/480 (62%)] Loss: -0.511
| Global Round : 1 | Local Epoch : 0 on device 63 | Images on device 63 [400/480 (83%)] Loss: -0.974
| Global Round : 1 | Local Epoch : 1 on device 63 | Images on device 63 [000/480 (0%)] Loss: -0.867
| Global Round : 1 | Local Epoch : 1 on device 63 | Images on device 63 [100/480 (21%)] Loss:

| Global Round : 1 | Local Epoch : 6 on device 2 | Images on device 2 [100/480 (21%)] Loss: -0.615
| Global Round : 1 | Local Epoch : 6 on device 2 | Images on device 2 [200/480 (42%)] Loss: -0.953
| Global Round : 1 | Local Epoch : 6 on device 2 | Images on device 2 [300/480 (62%)] Loss: -0.854
| Global Round : 1 | Local Epoch : 6 on device 2 | Images on device 2 [400/480 (83%)] Loss: -0.819
| Global Round : 1 | Local Epoch : 7 on device 2 | Images on device 2 [000/480 (0%)] Loss: -0.899
| Global Round : 1 | Local Epoch : 7 on device 2 | Images on device 2 [100/480 (21%)] Loss: -0.769
| Global Round : 1 | Local Epoch : 7 on device 2 | Images on device 2 [200/480 (42%)] Loss: -0.891
| Global Round : 1 | Local Epoch : 7 on device 2 | Images on device 2 [300/480 (62%)] Loss: -0.360
| Global Round : 1 | Local Epoch : 7 on device 2 | Images on device 2 [400/480 (83%)] Loss: -0.880
| Global Round : 1 | Local Epoch : 8 on device 2 | Images on device 2 [000/480 (0%)] Loss: -0.522
| Global Rou

| Global Round : 1 | Local Epoch : 3 on device 38 | Images on device 38 [200/480 (42%)] Loss: -0.863
| Global Round : 1 | Local Epoch : 3 on device 38 | Images on device 38 [300/480 (62%)] Loss: -0.749
| Global Round : 1 | Local Epoch : 3 on device 38 | Images on device 38 [400/480 (83%)] Loss: -0.604
| Global Round : 1 | Local Epoch : 4 on device 38 | Images on device 38 [000/480 (0%)] Loss: -0.810
| Global Round : 1 | Local Epoch : 4 on device 38 | Images on device 38 [100/480 (21%)] Loss: -0.835
| Global Round : 1 | Local Epoch : 4 on device 38 | Images on device 38 [200/480 (42%)] Loss: -0.759
| Global Round : 1 | Local Epoch : 4 on device 38 | Images on device 38 [300/480 (62%)] Loss: -0.783
| Global Round : 1 | Local Epoch : 4 on device 38 | Images on device 38 [400/480 (83%)] Loss: -0.847
| Global Round : 1 | Local Epoch : 5 on device 38 | Images on device 38 [000/480 (0%)] Loss: -0.787
| Global Round : 1 | Local Epoch : 5 on device 38 | Images on device 38 [100/480 (21%)] Loss:

| Global Round : 1 | Local Epoch : 9 on device 70 | Images on device 70 [400/480 (83%)] Loss: -0.839
| Global Round : 1 | Local Epoch : 0 on device 41 | Images on device 41 [000/480 (0%)] Loss: -0.674
| Global Round : 1 | Local Epoch : 0 on device 41 | Images on device 41 [100/480 (21%)] Loss: -0.721
| Global Round : 1 | Local Epoch : 0 on device 41 | Images on device 41 [200/480 (42%)] Loss: -0.714
| Global Round : 1 | Local Epoch : 0 on device 41 | Images on device 41 [300/480 (62%)] Loss: -0.716
| Global Round : 1 | Local Epoch : 0 on device 41 | Images on device 41 [400/480 (83%)] Loss: -0.865
| Global Round : 1 | Local Epoch : 1 on device 41 | Images on device 41 [000/480 (0%)] Loss: -0.689
| Global Round : 1 | Local Epoch : 1 on device 41 | Images on device 41 [100/480 (21%)] Loss: -0.593
| Global Round : 1 | Local Epoch : 1 on device 41 | Images on device 41 [200/480 (42%)] Loss: -0.796
| Global Round : 1 | Local Epoch : 1 on device 41 | Images on device 41 [300/480 (62%)] Loss:

In [ ]:
# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

In [ ]:
# Plot Loss curve
plt.title('Training Loss vs Communication rounds')
plt.plot(range(len(train_loss)), train_loss, color='r')
plt.ylabel('Training loss')
plt.xlabel('Communication Rounds')
plt.show()
plt.savefig('save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                   args.iid, args.local_ep, args.local_bs))

In [ ]:
# Plot Average Accuracy vs Communication rounds
plt.figure()
plt.title('Average Accuracy vs Communication rounds')
plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
plt.ylabel('Average Accuracy')
plt.xlabel('Communication Rounds')
plt.savefig('save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                   args.iid, args.local_ep, args.local_bs))